In [1]:
from utz.imports import *

In [80]:
cache = '/Volumes/T7/citibike'
env['AWS_PROFILE'] = 'rac'
Bucket = 'tripdata'

In [ ]:
from boto3 import client
s3 = client('s3')

In [3]:
resp = s3.list_objects_v2(Bucket=Bucket)
zips = pd.DataFrame(resp['Contents'])
zips

,Key,LastModified,ETag,Size,StorageClass
0,201306-citibike-tripdata.zip,2018-04-30 13:18:55+00:00,"""b520a12de58eea58a3586f89bfcfbd9d-2""",16785103,STANDARD
1,201307-201402-citibike-tripdata.zip,2017-01-18 22:23:25+00:00,"""7b3b260b2ab2e5349320121d04bd821c-22""",178262576,STANDARD
2,201307-citibike-tripdata.zip,2017-01-18 22:23:27+00:00,"""dd3e6fd5f91715b31eae72868086c08c-4""",27074629,STANDARD
3,201308-citibike-tripdata.zip,2017-01-18 22:23:27+00:00,"""2f661063576734f614b9f1d6bba0ec59-4""",32090869,STANDARD
4,201309-citibike-tripdata.zip,2017-01-18 22:23:27+00:00,"""a42f947db7bd14e423a7dbfbb11596a1-4""",33155593,STANDARD
...,...,...,...,...,...
144,JC-202005-citibike-tripdata.csv.zip,2020-06-05 16:09:40+00:00,"""a8a53f2ae57b29a9fe287a7e666a8c74""",655725,STANDARD
145,JC-202006-citibike-tripdata.csv.zip,2020-07-06 00:37:53+00:00,"""ebaaa7c1eb9467d04a7748feb6fc4f5b""",968484,STANDARD
146,JC-202007-citibike-tripdata.csv.zip,2020-08-12 02:13:32+00:00,"""4519a964d5b66156f9e661b7e78906ac""",1017946,STANDARD
147,JC-202008-citibike-tripdata.csv.zip,2020-09-04 19:37:33+00:00,"""bc110ea6ded190e5625213e05dce31fb""",1137151,STANDARD


In [56]:
rgx = r'^(?P<JC>JC-)?(?P<year>\d{4})(?P<month>\d{2})-citibike-tripdata(?P<csv>\.csv)?(?P<zip>\.zip)?$'

In [ ]:
from io import BytesIO
from zipfile import ZipFile

In [116]:
splitext('JC-201807-citibike-tripdata.csv.zip')

('JC-201807-citibike-tripdata.csv', '.zip')

In [117]:
splitext(splitext('JC-201807-citibike-tripdata.csv.zip')[0])

('JC-201807-citibike-tripdata', '.csv')

In [118]:
splitext('JC-201807-citibike-tripdata')

('JC-201807-citibike-tripdata', '')

In [103]:
def unzip(zip_path, error='warn'):
    name = basename(zip_path)
    m = match(rgx, name)
    if not m:
        msg = f'Unrecognized key: {name}'
        if error == 'warn':
            stderr.write(f'{msg}\n')
            return
        else:
            raise Exception(msg)
    assert name.endswith('.zip'), name
    csv_path = splitext(zip_path)[0]
    if not csv_path.endswith('.csv'):
        csv_path += '.csv'
    if not exists(csv_path):
        z = ZipFile(zip_path)
        names = z.namelist()
        print(f'{name}: zip names: {names}')
        [ name ] = [ f for f in names if f.endswith('.csv') and not f.startswith('_') ]
        with z.open(name,'r') as i, open(csv_path,'wb') as o:
            df = pd.read_csv(i)
            out = csv_path
            o.write(i.read())

In [99]:
zips = sorted(glob(f'{cache}/*.zip'))
[ basename(z) for z in zips ]

['201306-citibike-tripdata.zip',
 '201307-201402-citibike-tripdata.zip',
 '201307-citibike-tripdata.zip',
 '201308-citibike-tripdata.zip',
 '201309-citibike-tripdata.zip',
 '201310-citibike-tripdata.zip',
 '201311-citibike-tripdata.zip',
 '201312-citibike-tripdata.zip',
 '201401-citibike-tripdata.zip',
 '201402-citibike-tripdata.zip',
 '201403-citibike-tripdata.zip',
 '201404-citibike-tripdata.zip',
 '201405-citibike-tripdata.zip',
 '201406-citibike-tripdata.zip',
 '201407-citibike-tripdata.zip',
 '201408-citibike-tripdata.zip',
 '201409-citibike-tripdata.zip',
 '201410-citibike-tripdata.zip',
 '201411-citibike-tripdata.zip',
 '201412-citibike-tripdata.zip',
 '201501-citibike-tripdata.zip',
 '201502-citibike-tripdata.zip',
 '201503-citibike-tripdata.zip',
 '201504-citibike-tripdata.zip',
 '201505-citibike-tripdata.zip',
 '201506-citibike-tripdata.zip',
 '201507-citibike-tripdata.zip',
 '201508-citibike-tripdata.zip',
 '201509-citibike-tripdata.zip',
 '201510-citibike-tripdata.zip',
 '2

In [79]:
from joblib import delayed, Parallel
parallel = Parallel()

In [105]:
parallel(delayed(unzip)(f) for f in zips)

Unrecognized key: 201307-201402-citibike-tripdata.zip


201403-citibike-tripdata.zip: zip names: ['2014-03 - Citi Bike trip data.csv']
201404-citibike-tripdata.zip: zip names: ['2014-04 - Citi Bike trip data.csv']
201405-citibike-tripdata.zip: zip names: ['2014-05 - Citi Bike trip data.csv']
201406-citibike-tripdata.zip: zip names: ['2014-06 - Citi Bike trip data.csv']
201407-citibike-tripdata.zip: zip names: ['2014-07 - Citi Bike trip data.csv']
201408-citibike-tripdata.zip: zip names: ['2014-08 - Citi Bike trip data.csv']
201409-citibike-tripdata.zip: zip names: ['201409-citibike-tripdata.csv']
201410-citibike-tripdata.zip: zip names: ['201410-citibike-tripdata.csv']
201411-citibike-tripdata.zip: zip names: ['201411-citibike-tripdata.csv']
201412-citibike-tripdata.zip: zip names: ['201412-citibike-tripdata.csv']
201501-citibike-tripdata.zip: zip names: ['201501-citibike-tripdata.csv']
201502-citibike-tripdata.zip: zip names: ['201502-citibike-tripdata.csv']
201503-citibike-tripdata.zip: zip names: ['201503-citibike-tripdata.csv']
201504-c

Unrecognized key: JC-201708 citibike-tripdata.csv.zip


JC-201712-citibike-tripdata.csv.zip: zip names: ['JC-201712-citibike-tripdata.csv']
JC-201801-citibike-tripdata.csv.zip: zip names: ['JC-201801-citibike-tripdata.csv']
JC-201802-citibike-tripdata.csv.zip: zip names: ['JC-201802-citibike-tripdata.csv']
JC-201803-citibike-tripdata.csv.zip: zip names: ['JC-201803-citibike-tripdata.csv']
JC-201804-citibike-tripdata.csv.zip: zip names: ['JC-201804-citibike-tripdata.csv']
JC-201805-citibike-tripdata.csv.zip: zip names: ['JC-201805-citibike-tripdata.csv']
JC-201806-citibike-tripdata.csv.zip: zip names: ['JC-201806-citibike-tripdata.csv']
JC-201807-citibike-tripdata.csv.zip: zip names: ['JC-201807-citibike-tripdata.csv']
JC-201808-citibike-tripdata.csv.zip: zip names: ['JC-201808-citibike-tripdata.csv']
JC-201809-citibike-tripdata.csv.zip: zip names: ['JC-201809-citibike-tripdata.csv']
JC-201810-citibike-tripdata.csv.zip: zip names: ['JC-201810-citibike-tripdata.csv']
JC-201811-citibike-tripdata.csv.zip: zip names: ['JC-201811-citibike-tripdat

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

In [106]:
csvs = glob(f'{cache}/*.csv')
csvs

['/Volumes/T7/citibike/201309-citibike-tripdata.csv',
 '/Volumes/T7/citibike/201306-citibike-tripdata.csv',
 '/Volumes/T7/citibike/201307-citibike-tripdata.csv',
 '/Volumes/T7/citibike/201308-citibike-tripdata.csv',
 '/Volumes/T7/citibike/201310-citibike-tripdata.csv',
 '/Volumes/T7/citibike/201311-citibike-tripdata.csv',
 '/Volumes/T7/citibike/201312-citibike-tripdata.csv',
 '/Volumes/T7/citibike/201401-citibike-tripdata.csv',
 '/Volumes/T7/citibike/201402-citibike-tripdata.csv',
 '/Volumes/T7/citibike/201403-citibike-tripdata.csv',
 '/Volumes/T7/citibike/201404-citibike-tripdata.csv',
 '/Volumes/T7/citibike/201405-citibike-tripdata.csv',
 '/Volumes/T7/citibike/201406-citibike-tripdata.csv',
 '/Volumes/T7/citibike/201407-citibike-tripdata.csv',
 '/Volumes/T7/citibike/201408-citibike-tripdata.csv',
 '/Volumes/T7/citibike/201409-citibike-tripdata.csv',
 '/Volumes/T7/citibike/201410-citibike-tripdata.csv',
 '/Volumes/T7/citibike/201411-citibike-tripdata.csv',
 '/Volumes/T7/citibike/20141

In [107]:
csv = pd.read_csv(csvs[0])
csv

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender
0,1010,2013-09-01 00:00:02,2013-09-01 00:16:52,254,W 11 St & 6 Ave,40.735324,-73.998004,147,Greenwich St & Warren St,40.715422,-74.011220,15014,Subscriber,1974,1
1,1443,2013-09-01 00:00:09,2013-09-01 00:24:12,151,Cleveland Pl & Spring St,40.721816,-73.997203,497,E 17 St & Broadway,40.737050,-73.990093,19393,Customer,\N,0
2,1387,2013-09-01 00:00:16,2013-09-01 00:23:23,352,W 56 St & 6 Ave,40.763406,-73.977225,405,Washington St & Gansevoort St,40.739323,-74.008119,16160,Subscriber,1992,1
3,405,2013-09-01 00:00:18,2013-09-01 00:07:03,490,8 Ave & W 33 St,40.751551,-73.993934,459,W 20 St & 11 Ave,40.746745,-74.007756,14997,Subscriber,1973,1
4,270,2013-09-01 00:00:20,2013-09-01 00:04:50,236,St Marks Pl & 2 Ave,40.728419,-73.987140,393,E 5 St & Avenue C,40.722992,-73.979955,19609,Subscriber,1984,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1034354,93,2013-09-30 23:59:16,2013-10-01 00:00:49,360,William St & Pine St,40.707179,-74.008873,195,Liberty St & Broadway,40.709056,-74.010434,20594,Subscriber,1989,1
1034355,363,2013-09-30 23:59:21,2013-10-01 00:05:24,2003,1 Ave & E 18 St,40.734161,-73.980243,459,W 20 St & 11 Ave,40.746745,-74.007756,15213,Customer,\N,0
1034356,562,2013-09-30 23:59:30,2013-10-01 00:08:52,293,Lafayette St & E 8 St,40.730287,-73.990765,463,9 Ave & W 16 St,40.742065,-74.004432,17938,Subscriber,1967,1
1034357,1237,2013-09-30 23:59:40,2013-10-01 00:20:17,478,11 Ave & W 41 St,40.760301,-73.998842,318,E 43 St & Vanderbilt Ave,40.753202,-73.977987,15931,Customer,\N,0


In [114]:
sub('\.csv$', '.parquet', csvs[0])

'/Volumes/T7/citibike/201309-citibike-tripdata.parquet'

In [112]:
csvs[0].replace('.csv','.parquet')

'/Volumes/T7/citibike/201309-citibike-tripdata.parquet'

In [115]:
csv.to_parquet(sub('\.csv$', '.parquet', csvs[0]))

In [40]:
def get_month(year=None, month=None, key=None, jc=False, cache='citibike', overwrite=False, dask=True):
    if year is not None and month is not None:
        assert key is None
        key = f'{year}{"%02d" % int(month)}-citibike-tripdata.zip'
        if jc: key = f'JC-{key}'
    elif key is not None:
        assert year is None
        assert month is None

    cache_path = f'{cache}/{key}'
    if not exists(cache_path) or overwrite:
        with open(cache_path,'wb') as f:
            resp = s3.get_object(Bucket=Bucket, Key=key)
            f.write(resp['Body'].read())

    if dask:
        from dask import dataframe as dd
        read_csv = dd.read_csv
    else:
        read_csv = pd.read_csv

    with open(cache_path,'rb') as f:
        z = ZipFile(BytesIO(body))
        [ filename ] = [ 
            filename
            for filename in z.namelist
            if match(rgx, filename)
        ]
        with z.open(filename,'rb') as i,
        csv_str = decode(z.read(m.string))
        print(type(csv_bytes))
        df = read_csv(csv_str)

    return df

In [41]:
df_201306 = get_month(2013, 6)
df_201306

NameError: name 'decode' is not defined